In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import os
import re

In [2]:
#DATA EXTRACTION

In [3]:
driver = webdriver.Chrome("C:/Users/hp/Downloads/chromedriver_win32/chromedriver")
#os.makedirs('C://Users//hp//Documents//Black//all_files')


In [4]:
input_=pd.read_excel("Input.xlsx")
#print(input_)
for url_id,url in zip(input_['URL_ID'],input_['URL']):
    driver.get(url)
    content = driver.page_source
    soup = BeautifulSoup(content,'html.parser')
    for a in soup.findAll('header', attrs={'class':'td-post-title'}):
        title=a.find('h1', attrs={'class':'entry-title'})
    for a in soup.findAll('div', attrs={'class':'td-ss-main-content'}):
        info=a.find('div',attrs={'class':'td-post-content'})
    outputfile=open('C://Users//hp//Documents//Black//all_files//'+str(url_id)+'.txt','w',encoding="utf-8")
    outputfile.write(title.get_text()+'\n')
    outputfile.write(info.get_text())
    outputfile.close()
    #print(url)
    #print(url_id)

In [ ]:
#DATA ANALYSIS

In [5]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
stw=stopwords.words('english')

# stop words given in files
stopwords=[]
def stopwords_list(s):
    global stopwords
    s=s.read().replace('\n',' ').lower().replace('|','')
    stopwords.extend(word_tokenize(s))
stopwords_list(open('C://Users//hp//Documents//Black//StopWords//StopWords//StopWords_Generic.txt','r',encoding='utf8'))
stopwords_list(open('C://Users//hp//Documents//Black//StopWords//StopWords//StopWords_Auditor.txt','r',encoding='utf8'))
stopwords_list(open('C://Users//hp//Documents//Black//StopWords//StopWords//StopWords_Currencies.txt','r'))
stopwords_list(open('C://Users//hp//Documents//Black//StopWords//StopWords//StopWords_DatesandNumbers.txt','r',encoding='utf8'))
stopwords_list(open('C://Users//hp//Documents//Black//StopWords//StopWords//StopWords_GenericLong.txt','r',encoding='utf8'))
stopwords_list(open('C://Users//hp//Documents//Black//StopWords//StopWords//StopWords_Geographic.txt','r',encoding='utf8'))
stopwords_list(open('C://Users//hp//Documents//Black//StopWords//StopWords//StopWords_Names.txt','r',encoding='utf8'))
stopwords.extend([',','"','.','’','?'])
stopwords.extend(stw)
#print(stopwords)

#Positive and negative dictionary
def master_dictionary(s):
    #print(s.readlines())
    s=s.read().lower().replace('\n',' ')
    d=[]
    d.extend(word_tokenize(s))
    return d
POSITIVE_Dictionary=master_dictionary(open('C://Users//hp//Documents//Black//MasterDictionary//MasterDictionary//positive-words.txt','r',encoding='utf8'))
NEGATIVE_Dictionary=master_dictionary(open('C://Users//hp//Documents//Black//MasterDictionary//MasterDictionary//negative-words.txt','r'))

#POSITIVE_Dictionary
#NEGATIVE_Dictionary.readlines()
#NEGATIVE_Dictionary
#print(stw)

In [6]:
#output_=pd.read_excel("Output Data Structure.xlsx")
#print(output_.head())
input_=pd.read_excel("Input.xlsx")

url_id=[]
url=[]
positive_score=[]
negative_score=[]
polarity_score=[]
subjectivity_score=[]
avg_sent_length=[]
perc_of_complex_words=[]
fog_index=[]
avg_no_of_words_per_Sentence=[]
complex_word_count=[]
word_count=[]
SyllableCountPerWord=[]
personal_pronouns=[]
average_word_length=[]
for j in range(0,len(input_)):
    f = open("C://Users//hp//Documents//Black//all_files//"+str(input_['URL_ID'][j])+".txt",'r',encoding='utf8')
    txt = f.readlines()             # Using .read()
    f.close()
    text_words=[]
    without_stopwords=[]

####CLEANING - REMOVAL OF STOPWORDS
    sent_count=0
    personal_pronoun=0
    for sentences in txt:
        sent_count+=sentences.count(".")
        sent_count+=sentences.count("?")
        text_words.extend(word_tokenize(sentences))
        personal_pronoun+=len(re.findall("I", sentences))
        personal_pronoun+=len(re.findall('we', sentences))
        personal_pronoun+=len(re.findall('my', sentences))
        personal_pronoun+=len(re.findall('ours', sentences))
        personal_pronoun+=len(re.findall('us', sentences))
        #print(sentences,"*",sent_count)
    #print(len(text_words))
    #print("personal_pronoun",personal_pronoun)
    
    complex_words=0
    vowel=0
    word_unique={}
    char_len=0
    for w in text_words:
        if w.lower() not in stopwords:
            without_stopwords.append(w.lower())
            vowel=0
            char_len+=len(w)
            if w.lower() not in word_unique.keys():
                word_unique[w.lower()]=1
            else:
                word_unique[w.lower()]+=1
                
            for i in w.lower():
                if(i=='a' or i=='e' or i=='i' or i=='o' or i=='u'):
                    vowel+=1
            if w.lower().endswith('es') or w.lower().endswith('ed'):
                vowel-=1
            if vowel>2:
                #print(vowel,w.lower())
                complex_words+=1
                
    #print(len(without_stopwords))
    #print(len(word_unique))
    #print(char_len)
    
##### Creating a dictionary of Positive and Negative words

    POSI_words={}
    NEGI_words={}
    for w in without_stopwords:
        if w in POSITIVE_Dictionary:
            if w in POSI_words.keys():
                POSI_words[w]+=1
            else:
                POSI_words[w]=1
        elif w in NEGATIVE_Dictionary:
            if w in NEGI_words.keys():
                    NEGI_words[w]-=1
            else:
                NEGI_words[w]=int(-1)
    
    
    
    pos_sco=sum(POSI_words.values())
    neg_sco=sum(NEGI_words.values())*-1
    url.append((input_['URL'][j]))
    url_id.append((input_['URL_ID'][j]))
    positive_score.append(pos_sco)
    negative_score.append(neg_sco)
    polarity_score.append(round((pos_sco - neg_sco)/((pos_sco + neg_sco) +  0.000001),2))
    #(Positive Score – Negative Score)/ ((Positive Score + Negative Score) + 0.000001)
    subjectivity_score.append(round((pos_sco + neg_sco)/ ((len(without_stopwords)) + 0.000001),2))
    #Subjectivity Score = (Positive Score + Negative Score)/ ((Total Words after cleaning) + 0.000001)
    avg_sen_len=(len(word_unique)/sent_count)
    avg_sent_length.append(avg_sen_len) 
    #Average Sentence Length = the number of words / the number of sentences
    perc_cw=complex_words/len(word_unique)
    perc_of_complex_words.append(perc_cw) 
    #Percentage of Complex words = the number of complex words / the number of words 
    fog_index.append(0.4 * (avg_sen_len + perc_cw))
    #0.4 * (Average Sentence Length + Percentage of Complex words)
    avg_no_of_words_per_Sentence.append(round(len(without_stopwords)/sent_count,2))
    #Average Number of Words Per Sentence = the total number of words / the total number of sentences
    complex_word_count.append(complex_words)
    #COMPLEX WORD COUNT
    word_count.append(len(text_words))
    #word count
    SyllableCountPerWord.append(vowel)
    #Syllable Count Per Word
    personal_pronouns.append(personal_pronoun)
    #Personal Pronouns
    average_word_length.append(round(char_len/len(without_stopwords),2))
    #Average Word Length = Sum of the total number of characters in each word/Total number of words



In [7]:
output=pd.DataFrame({'URL_ID':url_id,'URL':url,'POSITIVE SCORE':positive_score,'NEGATIVE SCORE':negative_score,'POLARITY SCORE':polarity_score,
                    'SUBJECTIVITY SCORE':subjectivity_score,'AVG SENTENCE LENGTH':avg_sent_length,'PERCENTAGE OF COMPLEX WORDS':perc_of_complex_words,
                    'FOG INDEX':fog_index,'AVG NUMBER OF WORDS PER SENTENCE':avg_no_of_words_per_Sentence,'COMPLEX WORD COUNT':complex_word_count,
                    'WORD COUNT':word_count,'SYLLABLE PER WORD':SyllableCountPerWord,'PERSONAL PRONOUNS':personal_pronouns,'AVG WORD LENGTH':average_word_length})

In [8]:
output.head()

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,37,https://insights.blackcoffer.com/ai-in-healthc...,64,32,0.33,0.10,8.151899,0.860248,3.604859,12.29,554,2019,4,96,7.66
1,38,https://insights.blackcoffer.com/what-if-the-c...,58,38,0.21,0.17,4.421687,0.702997,2.049874,6.65,258,1652,2,70,7.22
2,39,https://insights.blackcoffer.com/what-jobs-wil...,65,37,0.27,0.12,6.082353,0.911025,2.797351,9.68,471,1928,2,90,7.66
3,40,https://insights.blackcoffer.com/will-machine-...,65,28,0.40,0.14,4.382979,0.762136,2.058046,6.89,314,1844,3,133,7.02
4,41,https://insights.blackcoffer.com/will-ai-repla...,57,25,0.39,0.10,6.137931,0.683521,2.728581,9.11,365,1989,4,97,7.15


In [9]:
output.to_excel("Output Data Structure Final.xlsx")


In [10]:
output_=pd.read_excel("Output Data Structure Final.xlsx")
output_

,Unnamed: 0,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,0,37,https://insights.blackcoffer.com/ai-in-healthc...,64,32,0.33,0.10,8.151899,0.860248,3.604859,12.29,554,2019,4,96,7.66
1,1,38,https://insights.blackcoffer.com/what-if-the-c...,58,38,0.21,0.17,4.421687,0.702997,2.049874,6.65,258,1652,2,70,7.22
2,2,39,https://insights.blackcoffer.com/what-jobs-wil...,65,37,0.27,0.12,6.082353,0.911025,2.797351,9.68,471,1928,2,90,7.66
3,3,40,https://insights.blackcoffer.com/will-machine-...,65,28,0.40,0.14,4.382979,0.762136,2.058046,6.89,314,1844,3,133,7.02
4,4,41,https://insights.blackcoffer.com/will-ai-repla...,57,25,0.39,0.10,6.137931,0.683521,2.728581,9.11,365,1989,4,97,7.15
5,5,42,https://insights.blackcoffer.com/man-and-machi...,43,22,0.32,0.12,5.833333,0.675325,2.603463,8.50,260,1503,3,67,7.22
6,6,43,https://insights.blackcoffer.com/in-future-or-...,22,11,0.33,0.10,4.297872,0.648515,1.978555,7.38,131,852,2,36,6.88
7,7,44,https://insights.blackcoffer.com/how-neural-ne...,22,11,0.33,0.10,4.297872,0.648515,1.978555,7.38,131,852,2,36,6.88
8,8,45,https://insights.blackcoffer.com/how-machine-l...,32,13,0.42,0.14,5.351351,0.762626,2.445591,8.62,151,829,3,36,6.84
9,9,46,https://insights.blackcoffer.com/deep-learning...,63,33,0.31,0.10,6.732558,0.803109,3.014267,10.92,465,2424,3,145,7.09


In [11]:
output_=output_.drop(['Unnamed: 0'],axis='columns')

In [12]:
output_

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,37,https://insights.blackcoffer.com/ai-in-healthc...,64,32,0.33,0.10,8.151899,0.860248,3.604859,12.29,554,2019,4,96,7.66
1,38,https://insights.blackcoffer.com/what-if-the-c...,58,38,0.21,0.17,4.421687,0.702997,2.049874,6.65,258,1652,2,70,7.22
2,39,https://insights.blackcoffer.com/what-jobs-wil...,65,37,0.27,0.12,6.082353,0.911025,2.797351,9.68,471,1928,2,90,7.66
3,40,https://insights.blackcoffer.com/will-machine-...,65,28,0.40,0.14,4.382979,0.762136,2.058046,6.89,314,1844,3,133,7.02
4,41,https://insights.blackcoffer.com/will-ai-repla...,57,25,0.39,0.10,6.137931,0.683521,2.728581,9.11,365,1989,4,97,7.15
5,42,https://insights.blackcoffer.com/man-and-machi...,43,22,0.32,0.12,5.833333,0.675325,2.603463,8.50,260,1503,3,67,7.22
6,43,https://insights.blackcoffer.com/in-future-or-...,22,11,0.33,0.10,4.297872,0.648515,1.978555,7.38,131,852,2,36,6.88
7,44,https://insights.blackcoffer.com/how-neural-ne...,22,11,0.33,0.10,4.297872,0.648515,1.978555,7.38,131,852,2,36,6.88
8,45,https://insights.blackcoffer.com/how-machine-l...,32,13,0.42,0.14,5.351351,0.762626,2.445591,8.62,151,829,3,36,6.84
9,46,https://insights.blackcoffer.com/deep-learning...,63,33,0.31,0.10,6.732558,0.803109,3.014267,10.92,465,2424,3,145,7.09
